In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFDSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)
tempF = InterfaceData(size=inodes, value=300.0)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = RelativeCovergenceMonitor(tol=1e-5)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 8000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FDSR%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FDSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve the wall temperature for convergence testing
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid flux interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        TempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = TempS.curr[s2f]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF)
                under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=10.
step=1, pc_iterations=8.
step=2, pc_iterations=7.
step=3, pc_iterations=6.
step=4, pc_iterations=6.
step=5, pc_iterations=6.
step=6, pc_iterations=5.
step=7, pc_iterations=5.
step=8, pc_iterations=5.
step=9, pc_iterations=5.
step=10, pc_iterations=5.
step=11, pc_iterations=5.
step=12, pc_iterations=5.
step=13, pc_iterations=5.
step=14, pc_iterations=5.
step=15, pc_iterations=5.
step=16, pc_iterations=5.
step=17, pc_iterations=3.
step=18, pc_iterations=3.
step=19, pc_iterations=3.
step=20, pc_iterations=3.
step=21, pc_iterations=3.
step=22, pc_iterations=3.
step=23, pc_iterations=3.
step=24, pc_iterations=3.
step=25, pc_iterations=3.
step=26, pc_iterations=3.
step=27, pc_iterations=3.
step=28, pc_iterations=3.
step=29, pc_iterations=3.
step=30, pc_iterations=3.
step=31, pc_iterations=3.
step=32, pc_iterations=3.
step=33, pc_iterations=3.
step=34, pc_iterations=3.
step=35, pc_iterations=3.
step=36, pc_iterations=3.
step=37, pc_iterations=3.
step=38, pc_iteration

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFDSR%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.93215808 0.93355619 0.93577729 0.93791848 0.93992946 0.94172388
 0.94339143 0.94492298 0.94634826 0.9477189  0.94899236 0.9502023
 0.95133992 0.95242815 0.95345345 0.95443368 0.95536158 0.9562522
 0.95710017 0.95790628 0.95867777 0.95941248 0.96011765 0.9607891
 0.96142877 0.96203963 0.96262625 0.9631859  0.96372244 0.96423475
 0.96472334 0.96519126 0.96563865 0.96606477 0.96647506 0.96686527
 0.96723717 0.96759181 0.96793189 0.96825542 0.96856424 0.96885855
 0.96913777 0.96940316 0.96965509 0.96989396 0.97011989 0.97033377
 0.97053493 0.97072464 0.970902   0.97106867 0.97122374 0.97136729
 0.97150055 0.97162217 0.97173402 0.97183449 0.97192454 0.97200417
 0.97207298 0.97213171 0.97217936 0.97221677 0.97224261 0.97225785
 0.9722624  0.97225487 0.97223647 0.97220582 0.97216288 0.972108
 0.97203945 0.97195859 0.97186345 0.97175404 0.9716283  0.97148818
 0.97133136 0.97115646 0.97096418 0.97075092 0.9705173  0.97026358
 0.96998403 0.9696806  0.96934764 0.96898583 0.96859166 0.9681588
 